In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!ls /content/drive/MyDrive/mrunals_project/converted_to_csv.zip

In [ ]:
!pip install joblib

In [ ]:
 !pip install scikit-learn-intelex

In [ ]:
# plugin to sklearn for supporting gpu and leveraging gpu while training
from sklearnex import patch_sklearn
patch_sklearn()


In [ ]:
# from google.colab import userdata
# userdata.get('secretName')
project_dir = '/content/drive/MyDrive/mrunals_project'
# project_dir = r'D:\dev\shriekdj\mrunal_related_project_for_workload_data'

In [ ]:
# import cudf as cpd
import pandas as pd
import zipfile
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
# from sklearn.ensemble import RandomForestRegressor

In [ ]:
zip_path = os.path.join(project_dir, 'converted_to_csv_again.zip')
zip_path

'/content/drive/MyDrive/mrunals_project/converted_to_csv_again.zip'

In [ ]:
import os
save_dir = os.path.join(project_dir, 'tf_models')
print(save_dir)
os.makedirs(save_dir, exist_ok=True)

/content/drive/MyDrive/mrunals_project/tf_models


In [ ]:
from typing import Literal
# Function to load all CSVs from a zip into a single DataFrame without needing to extract
def get_df(file, model_type: str):
  df = pd.read_csv(file)
  columns = list(df.columns)
  columns_only = list()
  columns_to_check = list()
  if model_type == 'resource':
    columns_to_check = ['Submit Time', 'Average CPU Time Used', 'Requested Number of Processors', 'Requested Time', 'User ID', 'Group ID', 'Allocated Processors']
  elif model_type == 'wait':
    columns_to_check = ['Allocated Processors', 'Requested Number of Processors', 'Submit Time', 'User ID', 'Wait Time']
  elif model_type == 'energy':
    columns_to_check = ['Used Memory', 'Run Time', 'Allocated Processors']

  for col in columns_to_check:
    if col in columns:
      columns_only.append(col)
  return df[columns_only]

def load_data_from_zip(zip_path: str, model_type: str):
    with zipfile.ZipFile(zip_path, 'r') as z:
        # List all file names in the zip that end with .csv
        csv_files = [f for f in z.namelist() if f.endswith('.csv')]
        # Load each CSV file directly from the zip and concatenate into one DataFrame
        df_list = [get_df(z.open(file), model_type) for file in csv_files]
        full_df = pd.concat(df_list, ignore_index=True)
        if model_type == 'resource':
          full_df.dropna(
              # subset=['Submit Time', 'Average CPU Time Used', 'Requested Number of Processors', 'Requested Time', 'User ID', 'Group ID', 'Allocated Processors'],
              inplace=True
            )
          return full_df
        elif model_type == 'wait':
          full_df.dropna(
              # subset=['Allocated Processors', 'Requested Number of Processors', 'Submit Time', 'User ID', 'Wait Time'],
              inplace=True
            )
          return full_df
        elif model_type == 'energy':
          full_df.dropna(
              # subset=['Used Memory', 'Run Time', 'Allocated Processors'],
              inplace=True
          )
          return full_df

In [ ]:
df = pd.DataFrame()
with zipfile.ZipFile(zip_path, 'r') as z:
    # List all file names in the zip that end with .csv
    csv_files = [f for f in z.namelist() if f.endswith('.csv')]
    # Load each CSV file directly from the zip and concatenate into one DataFrame
    for file in csv_files:
      df_temp = pd.read_csv(z.open(file), low_memory=False)
      df = pd.concat([df, df_temp], ignore_index=True)
    # df_list = [pd.read_csv(z.open(file), low_memory=False) for file in csv_files]

In [ ]:
df = load_data_from_zip(zip_path, 'resource')

In [ ]:
df

,Allocated Processors,Requested Number of Processors,Submit Time,User ID,Wait Time,Used Memory,Run Time,Allocated Processors
0,0.0,NaN,0.495143,93,0.012644,241664.0,0.006805,0.0
1,0.0,NaN,0.495143,93,0.012674,241664.0,0.006607,0.0
2,0.0,NaN,0.495143,93,0.012688,241664.0,0.006704,0.0
3,0.0,NaN,0.495143,93,0.012732,241664.0,0.006652,0.0
4,0.0,NaN,0.495143,93,0.012732,241664.0,0.006671,0.0
...,...,...,...,...,...,...,...,...
69825312,0.0,NaN,0.858831,211,0.025486,7168.0,0.000654,0.0
69825313,0.0,NaN,0.858831,211,0.025558,7168.0,0.000474,0.0
69825314,0.0,NaN,0.858831,211,0.025540,7168.0,0.000418,0.0
69825315,0.0,NaN,0.858831,211,0.025540,6144.0,0.000584,0.0


In [ ]:
# df = pd.concat(df, ignore_index=True)
del df_cleaned

In [ ]:
df.columns

Index(['Allocated Processors', 'Requested Number of Processors', 'Submit Time',
       'User ID', 'Wait Time', 'Used Memory', 'Run Time',
       'Allocated Processors'],
      dtype='object')

## Resource Requirement Prediction Model (using Gradient Boosting Regression)

This model will predict the amount of resources needed (e.g., Allocated Processors, Used Memory) based on the job characteristics.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
df_cleaned = load_data_from_zip(zip_path, 'resource')


In [ ]:
# Proceed with your existing feature selection and model training steps
X = df_cleaned[['Submit Time', 'Average CPU Time Used', 'Requested Number of Processors', 'Requested Time', 'User ID', 'Group ID']]
y = df_cleaned['Allocated Processors']

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize and train the model with cpu
model_resource = GradientBoostingRegressor(random_state=42)
model_resource.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

In [ ]:
# Predict and evaluate the model
predictions_resource = model_resource.predict(X_test)
mse_resource = mean_squared_error(y_test, predictions_resource)
print('RMSE for Resource Requirement Model:', np.sqrt(mse_resource))

RMSE for Resource Requirement Model: 0.02280472965140303


In [ ]:
import joblib
joblib.dump(
    model_resource,
    os.path.join(project_dir, 'model_resource.joblib')
)

In [ ]:
# Define the neural network model
def create_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
# Create and train the model using the TPUStrategy
with strategy.scope():
    model_resource = create_model()
    model_resource.fit(X_train, y_train, epochs=10, validation_split=0.2)

NameError: name 'strategy' is not defined

In [ ]:
predictions_resource = model_resource.predict(X_test)
mse_resource = tf.keras.losses.MeanSquaredError()(y_test, predictions_resource)
print('RMSE for Resource Allocation Model:', tf.sqrt(mse_resource).numpy())

In [ ]:
model_resource.save(os.path.join(save_dir, 'model_resource.h5'))

## Wait Time Optimization Model ( Using Random Forest Regressor )

This model predicts the wait time for a job to start based on its resource requirements and other features.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

df_wait_cleaned = load_data_from_zip(zip_path, 'wait')
# del df

In [1]:
# Assume 'Wait Time' prediction needs features about the job and the system status
X_wait = df_wait_cleaned[['Allocated Processors', 'Requested Number of Processors', 'Submit Time', 'User ID']]
y_wait = df_wait_cleaned['Wait Time'].astype(float)

NameError: name 'df_wait_cleaned' is not defined

In [ ]:
# Splitting data
X_train_wait, X_test_wait, y_train_wait, y_test_wait = train_test_split(X_wait, y_wait, test_size=0.2, random_state=42)

In [ ]:
del X_wait
del y_wait
del df_wait_cleaned

In [ ]:
# Model for Wait Time
# model_wait = RandomForestRegressor(n_estimators=100, random_state=42)
model_wait = GradientBoostingRegressor(random_state=42)
model_wait.fit(X_train_wait, y_train_wait)

GradientBoostingRegressor(random_state=42)

In [ ]:
# Predicting and Evaluating
predictions_wait = model_wait.predict(X_test_wait)
mse_wait = mean_squared_error(y_test_wait, predictions_wait)
print('RMSE for Wait Time Model:', np.sqrt(mse_wait))

RMSE for Wait Time Model: 0.010170133360804629


In [ ]:
import joblib
joblib.dump(model_wait, os.path.join(project_dir, 'model_wait.joblib'))

['/content/drive/MyDrive/mrunals_project/model_wait.joblib']

In [ ]:
import joblib
joblib.dump(X_train_wait, 'X_train_wait.joblib')
joblib.dump(X_test_wait, 'X_test_wait.joblib')
joblib.dump(y_train_wait, 'y_train_wait.joblib')
joblib.dump(y_test_wait, 'y_test_wait.joblib')

['y_test_wait.joblib']

In [ ]:
# Define the neural network model for wait time prediction
def create_wait_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[X_train_wait.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
# Train the model on TPU
with strategy.scope():
    model_wait = create_wait_model()
    model_wait.fit(X_train_wait, y_train_wait, epochs=10, validation_split=0.2)

In [ ]:
predictions_wait = model_wait.predict(X_test_wait)
mse_wait = tf.keras.losses.MeanSquaredError()(y_test_wait, predictions_wait)
print('RMSE for Wait Time Model:', tf.sqrt(mse_wait).numpy())

In [ ]:
model_wait.save(os.path.join(save_dir, 'model_wait.h5'))

## Energy Consumption Optimization Model ( Graadient Boosting Regressor )

Predicts energy consumption based on system usage and job characteristics.

In [ ]:
df_energy_cleaned = load_data_from_zip(zip_path, model_type='energy')
# df_energy_cleaned = df_energy_cleaned.dropna(subset=['Used Memory', 'Run Time', 'Allocated Processors'])
# df_energy_cleaned = df.dropna(subset=['Used Memory', 'Run Time', 'Allocated Processors'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
# Simulated feature for Energy Consumption based on existing features
df_energy_cleaned['Energy Consumption'] = df_energy_cleaned['Used Memory'] * 0.5  # Placeholder for actual energy consumption logic

In [ ]:
X_energy = df_energy_cleaned[['Used Memory', 'Run Time', 'Allocated Processors']]
y_energy = df_energy_cleaned['Energy Consumption']

In [ ]:
# Splitting data
X_train_energy, X_test_energy, y_train_energy, y_test_energy = train_test_split(
    X_energy,
    # df_energy_cleaned[['Used Memory', 'Run Time', 'Allocated Processors']],
    y_energy,
    # df_energy_cleaned['Energy Consumption'],
    test_size=0.2, random_state=42
)

In [ ]:
del X_energy
del y_energy
del df_energy_cleaned

In [ ]:
# Energy model
model_energy = RandomForestRegressor(n_estimators=100,random_state=42)
model_energy.fit(X_train_energy, y_train_energy)

In [ ]:
# Prediction and evaluation
predictions_energy = model_energy.predict(X_test_energy)
mse_energy = mean_squared_error(y_test_energy, predictions_energy)
print('RMSE for Energy Consumption Model:', np.sqrt(mse_energy))

RMSE for Energy Consumption Model: 6952.187715214187


In [ ]:
!pip install joblib

In [ ]:
import joblib
joblib.dump(model_energy, os.path.join(project_dir, 'model_energy.joblib'))

['/content/drive/MyDrive/mrunals_project/model_energy.joblib']

In [ ]:
joblib.dump(X_train_energy, 'X_train_energy.joblib')
joblib.dump(X_test_energy, 'X_test_energy.joblib')
joblib.dump(y_train_energy, 'y_train_energy.joblib')
joblib.dump(y_test_energy, 'y_test_energy.joblib')

['y_test_energy.joblib']

In [ ]:
with zipfile.ZipFile(os.path.join(project_dir, 'model_wait_training_and_test_files.zip'), 'w', zipfile.ZIP_DEFLATED) as zip:
    # Writing each file one by one
    for file in ['X_train_wait.joblib', 'X_test_wait.joblib', 'y_train_wait.joblib', 'y_test_wait.joblib']:
#oblib', 'y_train_energy.joblib', 'y_test_energy.joblib'
        print(file)
        zip.write(os.path.join(os.getcwd(), file), arcname=file)

X_train_wait.joblib
X_test_wait.joblib
y_train_wait.joblib
y_test_wait.joblib


In [ ]:
# Define the neural network model for energy consumption prediction
def create_energy_model():
    model = keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=[X_train_energy.shape[1]]),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
# Train the model on TPU
with strategy.scope():
    model_energy = create_energy_model()
    model_energy.fit(X_train_energy, y_train_energy, epochs=10, validation_split=0.2)

In [ ]:
predictions_energy = model_energy.predict(X_test_energy)
mse_energy = tf.keras.losses.MeanSquaredError()(y_test_energy, predictions_energy)
print('RMSE for Energy Consumption Model:', tf.sqrt(mse_energy).numpy())

In [ ]:
model_energy.save(os.path.join(save_dir, 'model_energy.h5'))

## Loading the Models

If you need to load these models back into another session, you can easily do so with the `load_model` function from `keras.models`. Here's an example of how you would load one of the saved models:

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the file
model_resource = load_model(os.path.join(save_dir, 'model_resource.h5'))

# Use the model for prediction, evaluation, etc.